### 1. Set Up the Environment in Google Colab

####a. Install Required Libraries

In [ ]:
# Install required libraries
!pip install streamlit
!pip install pyngrok
!pip install transformers
!pip install torch
!pip install yake
!pip install gensim
!pip install nltk
!pip install textblob
!pip install wordcloud
!pip install plotly
!pip install networkx
!pip install psycopg2 SQLAlchemy





####b. Download NLTK Data

Download the required NLTK data for text processing.

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

###2. Set Up ngrok for Hosting the Streamlit App
####a. Install and Authenticate ngrok

In [ ]:
!pip install --upgrade pyngrok


In [ ]:
from pyngrok import ngrok

ngrok_auth_token = "your_ngrok_token"
!ngrok authtoken {ngrok_auth_token}


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


####c. Create app.py

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import sqlite3
from wordcloud import WordCloud, STOPWORDS
import plotly.express as px
import plotly.graph_objs as go
import torch
from transformers import pipeline
import yake
from gensim import corpora, models
import matplotlib.pyplot as plt
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
import networkx as nx
from io import StringIO
import os

# Ensure NLTK data is downloaded
nltk.download('punkt')
nltk.download('stopwords')

# Initialize Hugging Face pipelines
@st.cache(allow_output_mutation=True)
def load_pipelines():
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
    sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
    return summarizer, sentiment_analyzer

summarizer, sentiment_analyzer = load_pipelines()

# Initialize SQLite database
conn = sqlite3.connect('text_data.db')
c = conn.cursor()

# Create table if not exists
c.execute('''
    CREATE TABLE IF NOT EXISTS texts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        content TEXT,
        summary TEXT,
        themes TEXT,
        sentiment TEXT
    )
''')
conn.commit()

# Function to insert data into the database
def insert_data(title, content, summary, themes, sentiment):
    c.execute('''
        INSERT INTO texts (title, content, summary, themes, sentiment)
        VALUES (?, ?, ?, ?, ?)
    ''', (title, content, summary, themes, sentiment))
    conn.commit()

# Function to retrieve data
def get_all_data():
    c.execute('SELECT * FROM texts')
    data = c.fetchall()
    return data

# Function for keyword extraction using YAKE
def extract_keywords(text, max_keywords=5):
    kw_extractor = yake.KeywordExtractor(lan="en", n=1, dedupLim=0.9, top=max_keywords, features=None)
    keywords = kw_extractor.extract_keywords(text)
    return ", ".join([kw[0] for kw in keywords])

# Function for theme extraction using LDA (Gensim)
def extract_topics(texts, num_topics=3):
    # Tokenization and cleaning
    stop_words = set(stopwords.words('english'))
    processed_texts = []
    for doc in texts:
        tokens = nltk.word_tokenize(doc.lower())
        tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
        processed_texts.append(tokens)

    # Create dictionary and corpus
    dictionary = corpora.Dictionary(processed_texts)
    corpus = [dictionary.doc2bow(text) for text in processed_texts]

    # LDA model
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

    # Extract topics
    topics = lda_model.print_topics(num_words=3)
    theme_list = [topic[1] for topic in topics]
    return "; ".join(theme_list)

# Streamlit App Layout
st.title("AI-Enhanced Interactive Textual Insights and Visualization Platform")
st.write("Upload your textual data, analyze it with AI, and visualize the insights.")

# Sidebar for Uploading Text
st.sidebar.header("Upload Text Data")
uploaded_file = st.sidebar.file_uploader("Choose a text file (TXT or CSV)", type=["txt", "csv"])

if uploaded_file is not None:
    # Read the uploaded file
    if uploaded_file.type == "text/csv":
        data = pd.read_csv(uploaded_file)
        text_column = st.sidebar.selectbox("Select text column", data.columns)
        texts = data[text_column].astype(str).tolist()
    else:
        data = StringIO(uploaded_file.getvalue().decode("utf-8"))
        texts = data.read().split('\n')

    st.sidebar.write(f"Number of texts uploaded: {len(texts)}")

    # Input for Title
    title = st.sidebar.text_input("Enter a title for this batch of texts", "Uploaded Texts")

    if st.sidebar.button("Analyze and Save"):
        with st.spinner("Analyzing texts with AI..."):
            summaries = []
            themes_list = []
            sentiments = []

            for text in texts:
                if text.strip() == "":
                    continue
                # Generate Summary
                try:
                    summary = summarizer(text, max_length=60, min_length=25, do_sample=False)[0]['summary_text']
                except Exception as e:
                    summary = "Summary unavailable."
                summaries.append(summary)

                # Extract Themes using Keyword Extraction and Topic Modeling
                keywords = extract_keywords(text)
                themes_list.append(keywords)

                # Perform Sentiment Analysis
                try:
                    sentiment = sentiment_analyzer(text)[0]['label']
                except Exception as e:
                    sentiment = "Unknown"
                sentiments.append(sentiment)

                # Insert into database
                insert_data(title, text, summary, keywords, sentiment)

        st.success("Analysis complete and data saved to database!")

# Display Stored Data
st.header("Stored Textual Data")
data = get_all_data()
if data:
    df = pd.DataFrame(data, columns=['ID', 'Title', 'Content', 'Summary', 'Themes', 'Sentiment'])
    st.dataframe(df)

    # Visualization Options
    st.header("Visualizations")

    # Sentiment Distribution
    st.subheader("Sentiment Distribution")
    sentiment_counts = df['Sentiment'].value_counts().reset_index()
    sentiment_counts.columns = ['Sentiment', 'Count']
    fig1 = px.pie(sentiment_counts, names='Sentiment', values='Count', title='Sentiment Distribution')
    st.plotly_chart(fig1)

    # Themes Word Cloud
    st.subheader("Themes Word Cloud")
    all_themes = ' '.join(df['Themes'].tolist())
    stop_words = set(stopwords.words('english'))
    wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=stop_words, collocations=False).generate(all_themes)
    fig2, ax = plt.subplots(figsize=(15, 7.5))
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.axis('off')
    st.pyplot(fig2)

    # Thematic Relationship Network
    st.subheader("Thematic Relationship Network")
    # Split themes and create edges based on co-occurrence in texts
    theme_edges = []
    for themes in df['Themes']:
        theme_list = [theme.strip().lower() for theme in themes.split(',') if theme.strip() != '']
        for i in range(len(theme_list)):
            for j in range(i+1, len(theme_list)):
                theme_edges.append((theme_list[i], theme_list[j]))

    # Count edge frequencies
    edge_df = pd.DataFrame(theme_edges, columns=['Theme1', 'Theme2'])
    edge_counts = edge_df.groupby(['Theme1', 'Theme2']).size().reset_index(name='Count')

    if not edge_counts.empty:
        # Create a graph
        G = nx.from_pandas_edgelist(edge_counts, 'Theme1', 'Theme2', ['Count'])

        # Create positions
        pos = nx.spring_layout(G, k=0.5, iterations=20)

        # Create edge traces
        edge_x = []
        edge_y = []
        for edge in G.edges():
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])

        edge_trace = go.Scatter(
            x=edge_x, y=edge_y,
            line=dict(width=0.5, color='#888'),
            hoverinfo='none',
            mode='lines')

        # Create node traces
        node_x = []
        node_y = []
        for node in G.nodes():
            x, y = pos[node]
            node_x.append(x)
            node_y.append(y)

        node_trace = go.Scatter(
            x=node_x, y=node_y,
            mode='markers',
            hoverinfo='text',
            marker=dict(
                showscale=True,
                colorscale='YlGnBu',
                size=[5 + G.degree(node)*2 for node in G.nodes()],
                color=[G.degree(node) for node in G.nodes()],
                colorbar=dict(
                    thickness=15,
                    title='Degree',
                    xanchor='left',
                    titleside='right'
                ),
                line_width=2))

        node_text = []
        for node in G.nodes():
            node_text.append(f"{node} (Degree: {G.degree(node)})")
        node_trace.text = node_text

        # Create figure
        fig3 = go.Figure(data=[edge_trace, node_trace],
                     layout=go.Layout(
                        title='Thematic Relationship Network',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20,l=5,r=5,t=40),
                        annotations=[ dict(
                            text="AI-Enhanced Interactive Textual Insights and Visualization Platform",
                            showarrow=False,
                            xref="paper", yref="paper",
                            x=0.005, y=-0.002 ) ],
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                        )

        st.plotly_chart(fig3)
    else:
        st.write("Not enough data to generate Thematic Relationship Network.")

    # Interactive Q&A with AI
    st.header("Ask Questions About Your Texts")
    user_question = st.text_input("Enter your question:")
    if st.button("Get Answer"):
        if user_question.strip() == "":
            st.write("Please enter a valid question.")
        else:
            with st.spinner("Generating answer..."):
                # Since we are not using a generative model, we'll use keyword matching or predefined responses
                # For more advanced Q&A, consider integrating with a local LLM if possible
                # Here, we'll provide a placeholder response
                st.write("**Answer:** This feature requires a generative model. Currently, it's not implemented.")
else:
    st.write("Upload a text file to begin analysis.")


Overwriting app.py


###b. Function to run Streamlit app

In [ ]:
from pyngrok import ngrok
from threading import Thread
import os
import time
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def manage_ngrok_tunnels(max_tunnels=2):
    """
    Manages ngrok tunnels by ensuring that the number of active tunnels does not exceed max_tunnels.
    If the limit is reached, the oldest tunnels are terminated to make room for new ones.
    """
    tunnels = ngrok.get_tunnels()
    active_tunnels = len(tunnels)

    if active_tunnels >= max_tunnels:
        # Number of tunnels to terminate
        num_to_kill = active_tunnels - max_tunnels + 1
        logger.info(f"Active tunnels ({active_tunnels}) >= max_tunnels ({max_tunnels}). Terminating {num_to_kill} tunnel(s).")

        for i in range(num_to_kill):
            tunnel = tunnels[i]
            try:
                ngrok.disconnect(tunnel.public_url)
                logger.info(f"Terminated tunnel: {tunnel.public_url}")
            except Exception as e:
                logger.error(f"Failed to terminate tunnel {tunnel.public_url}: {e}")

def run_app():
    """
    Runs the Streamlit app.
    """
    os.system("streamlit run app.py --server.port 8501")

# Manage existing tunnels before starting a new one
manage_ngrok_tunnels(max_tunnels=2)

# Start the Streamlit app in a separate thread
app_thread = Thread(target=run_app)
app_thread.start()

# Allow some time for the Streamlit app to start
time.sleep(5)

# Create a new ngrok tunnel for the Streamlit app
try:
    public_url = ngrok.connect(addr=8501, proto="http")
    logger.info(f"🔗 Streamlit app is live at: {public_url}")
    print(f"🔗 Streamlit app is live at: {public_url}")
except Exception as e:
    logger.error(f"Failed to create ngrok tunnel: {e}")
    print(f"❌ Failed to create ngrok tunnel: {e}")


🔗 Streamlit app is live at: NgrokTunnel: "https://5dd2-34-106-243-49.ngrok-free.app" -> "http://localhost:8501"
